In [597]:



#FIRST MAKE IT FOR COMPANIES ONLY IN THE S&P, THEN CHANGE THE CODE TO DO ALL COMPANIES IN NYSE AND NASDAQ

# COMPARE MONEY SPENT ON LOBBYING VS MARKET CAP



# Does money spent on lobbying have any correlation with the yearly returns of a company's stock?
Public companies spend millions (if not billions) of dollars each year to lobby Congress and other federal agencies. Does spending such a large amount of money on lobbying work? More than likely. Can this be reflected somehow in the returns that the company's stock provides to its investors? Will investing in a company that spends large amounts on lobbying yield better returns than investing in one that does not? That is the purpose of this exploratory data anlysis.


**Start with background info into the problem and why it is important to solve.
Give basic info on the dataset, how it was collected, and what the notebook is trying to achieve/solve.

### Importing needed libraries / modules

In [609]:
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from crpapi import CRP

### Scraping the current companies in the S&P500 from wikipedia

In [599]:
wikiurl = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
stockstable=soup.find('table',{'class':"wikitable"})

# convert list to dataframe
df = pd.read_html(str(stockstable))
df = pd.DataFrame(df[0])

# let's take a look at the data
df.head(10)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


Don't need the following columns: SEC filings, Headquarters Location, or Date Founded

In [600]:
df = df.drop(columns=["SEC filings", "Headquarters Location", "Founded"], axis=1)

Looking for issues in the dataset...

In [601]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Symbol             505 non-null    object
 1   Security           505 non-null    object
 2   GICS Sector        505 non-null    object
 3   GICS Sub-Industry  505 non-null    object
 4   Date first added   460 non-null    object
 5   CIK                505 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 23.8+ KB


All looks good. I will leave in the "Date first added" column, it may come in handy later when scraping the returns or money spent on lobbying each year


### Finding the returns per year of every company in the dataframe

First I will add the required columns to the dataframe:

In [602]:
currentMonth = datetime.now().month
currentYear = datetime.now().year

for year in range(currentYear - 1, currentYear - int(period), -1):
    df[year] = np.nan
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Symbol             505 non-null    object 
 1   Security           505 non-null    object 
 2   GICS Sector        505 non-null    object 
 3   GICS Sub-Industry  505 non-null    object 
 4   Date first added   460 non-null    object 
 5   CIK                505 non-null    int64  
 6   2021               0 non-null      float64
 7   2020               0 non-null      float64
 8   2019               0 non-null      float64
 9   2018               0 non-null      float64
 10  2017               0 non-null      float64
 11  2016               0 non-null      float64
 12  2015               0 non-null      float64
 13  2014               0 non-null      float64
 14  2013               0 non-null      float64
 15  2012               0 non-null      float64
 16  2011               0 non-n

Depending on when this jupyter notebook is ran, the TD Ameritrade API will return an (period * 12) amount of candles. This means that unless it is currently December, the earliest year will have incomplete candle data. I will be removing this year with incomplete data to avoid skewing our final results. Also, we do not want to include the current year since the data for that is obviously also incomplete.

Setting up the parameters to make the GET request to the TD Ameritrade API:

In [603]:
template_url = "https://api.tdameritrade.com/v1/marketdata"
apikey = ""
periodType = "year"
period = "20" # Show last 20 years. If periodType == "year", period can only be 1,2,3,5,10,15,20
frequencyType = "monthly" #Type of frequency with which a new candle is formed
PARAMS = {'apikey':apikey, 'periodType':periodType, 'period':period, 'frequencyType':frequencyType}

Calculating the return for every available year for each company, and including it in the dataframe:

In [604]:
for index,row in df.iterrows():
    stockSymbol = row['Symbol']
    finalURL = template_url + "/" + stockSymbol + "/" + "pricehistory"
    r = requests.get(url = finalURL, params = PARAMS)
    data = r.json()
    # Now data["candles"] is a list with each element being a dictionary describing a monthly candle.
    # We want to use the keys "open" and "close"
    
    # Eliminate the candles from years with incomplete data:
    
    # First for the current year
    for candle in range(currentMonth):
        data["candles"].pop(0)
        
    unecessaryMonths = len(data["candles"]) % 12
    
    # Now for the first incomplete year
    for candle in range(unecessaryMonths):
        data["candles"].pop(-1)
    
    # Find the returns for each year for the stockSymbol
    yearsWithData = len(data["candles"]) // 12
    returns = dict()
    currentYear = datetime.now().year - 1
    janCounter = 0
    decCounter = 11
    
    
    for year in range(yearsWithData):
        returns[currentYear] = (data["candles"][decCounter]["close"] - data["candles"][janCounter]["open"])\
        * 100 / data["candles"][janCounter]["open"]
        
        janCounter += 12
        decCounter += 12
        currentYear -= 1
        
    # Append the returns for the stock to the dataframe
    for key,val in returns.items():
        df.at[index, key] = val
        
    # Need to wait for some time before going back into the for loop,
    # otherwise TD Ameritrade API reaches a limit on transactions per second
    time.sleep(1)
    

MMM
AOS
ABT
ABBV
ABMD
ACN
ATVI
ADM
ADBE
ADP
AAP
AES
AFL
A
AIG
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMD
AEE
AAL
AEP
AXP
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
APA
AAPL
AMAT
APTV
ANET
AIZ
T
ATO
ADSK
AZO
AVB
AVY
BKR
BLL
BAC
BBWI
BAX
BDX
WRB
BRK.B
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
CHRW
CDNS
CZR
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CERN
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CTVA
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DIS
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ENPH
ETR
EOG
EPAM
EFX
EQIX
EQR
ESS
EL
ETSY
RE
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FAST
FRT
FDX
FITB
FRC
FE
FIS
FISV
FLT
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
AJG
GRMN
IT
GE
GNRC
GD
GIS
GPC
GILD
GL
GPN
GM
GS
GWW
HAL
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HS

Now let's check that the dataframe was filled as intended and that we have enough data.

In [606]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Symbol             505 non-null    object 
 1   Security           505 non-null    object 
 2   GICS Sector        505 non-null    object 
 3   GICS Sub-Industry  505 non-null    object 
 4   Date first added   460 non-null    object 
 5   CIK                505 non-null    int64  
 6   2021               505 non-null    float64
 7   2020               503 non-null    float64
 8   2019               500 non-null    float64
 9   2018               498 non-null    float64
 10  2017               493 non-null    float64
 11  2016               489 non-null    float64
 12  2015               482 non-null    float64
 13  2014               475 non-null    float64
 14  2013               466 non-null    float64
 15  2012               457 non-null    float64
 16  2011               451 non

The decreasing non-null count of each column makes sense.

### Finding the amount of money spent on lobbying for every company in the dataframe

For this, we will be using the OpenSecrets API. All lobbying numbers are credited to them and can be found in https://www.opensecrets.org/federal-lobbying by looking for each specific company.

This is the process that will be followed:
1. Find if the company 

In [614]:
crp = CRP("")
test = crp.orgs.get("Abiomed")
print(test)

{'@attributes': {'orgid': 'D000023733', 'orgname': 'Abiomed Inc'}}


In [615]:
df["orgid"] = ""

In [616]:
df.head(10)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Date first added,CIK,2021,2020,2019,2018,...,2011,2010,2009,2008,2007,2006,2005,2004,2003,orgid
0,MMM,3M,Industrials,Industrial Conglomerates,1976-08-09,66740,2.568588,39.572027,-19.666667,11.699627,...,22.466122,32.176802,7.593613,12.162941,18.870618,-5.938606,-11.316893,-11.096546,27.050019,
1,AOS,A. O. Smith,Industrials,Building Products,2017-07-26,91142,-9.775635,12.930734,-15.979872,73.438092,...,48.202614,36.508811,45.208796,21.230050,27.862899,4.431497,-19.577080,-1.833333,52.153716,
2,ABT,Abbott,Health Care,Health Care Equipment,1964-03-31,1800,14.915964,-6.620846,20.122551,-11.415804,...,13.509362,16.957392,19.184070,-20.425101,23.658102,25.159040,38.710209,7.894737,26.038269,
3,ABBV,AbbVie,Health Care,Pharmaceuticals,2012-12-31,1551152,28.628629,13.405657,-2.325581,15.164369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,ABMD,Abiomed,Health Care,Health Care Equipment,2018-05-31,815094,-36.308806,135.140187,-31.159420,50.813953,...,-6.097561,16.173752,160.419968,64.543812,30.617081,182.454081,-35.997543,-8.183511,28.461475,
5,ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06,1467373,-4.789474,52.836485,-17.623547,24.757709,...,20.354574,12.059884,19.348872,15.909556,8.815766,32.119205,14.126004,16.171617,37.431235,
6,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,2015-08-31,718877,-34.003306,85.593869,38.098301,-9.032774,...,18.932444,56.491228,8.419167,62.418033,45.049131,32.110092,-37.367304,58.521303,26.494367,
7,ADM,ADM,Consumer Staples,Agricultural Products,1981-07-29,7084,2.142857,30.889236,27.794381,91.732466,...,17.132988,29.316916,8.873335,-11.948265,-3.745057,10.486982,-10.897576,-3.506651,51.122195,
8,ADBE,Adobe,Information Technology,Application Software,1997-05-05,796343,12.093674,46.318439,23.372143,5.636743,...,43.000628,59.982313,12.373422,17.432880,48.821549,72.024272,22.236051,45.321315,34.691812,
9,ADP,ADP,Information Technology,Data Processing & Outsourced Services,1981-03-31,8670,-22.285977,24.052135,-0.308788,7.719716,...,24.318469,15.182334,14.810965,15.545214,11.369565,31.535595,24.626866,-11.174084,33.105482,
